In [1]:
import pandas as pd
import json

In [47]:
df_tydiqa = pd.read_json('../data/raw/TyDiQA/tydiqa-goldp-v1.1-train-indonesian.json')
df_tydiqa_dev = pd.read_json('../data/raw/TyDiQA/tydiqa-goldp-v1.1-dev-indonesian.json')

df_tydiqa['title'] = df_tydiqa['data'].apply(lambda x: x['title'])
df_tydiqa_dev['title'] = df_tydiqa_dev['data'].apply(lambda x: x['title'])
df_tydiqa['data'] = df_tydiqa['data'].apply(lambda x: x['paragraphs'])
df_tydiqa_dev['data'] = df_tydiqa_dev['data'].apply(lambda x: x['paragraphs'])
df_tydiqa = df_tydiqa.rename(columns={'data':'paragraphs'}).drop(columns=['version'])
df_tydiqa_dev = df_tydiqa_dev.rename(columns={'data':'paragraphs'}).drop(columns=['version'])
df_tydiqa = df_tydiqa[['title', 'paragraphs']]
df_tydiqa_dev = df_tydiqa_dev[['title', 'paragraphs']]

print(df_tydiqa.head(8))
print(df_tydiqa.shape)
print(df_tydiqa_dev.head(8))
print(df_tydiqa_dev.shape)

                            title  \
33713        Ernest Douwes Dekker   
33714            Balai Kota Seoul   
33715  Perserikatan Bangsa-Bangsa   
33716               James Hepburn   
33717                 Jamie Vardy   
33718             John F. Kennedy   
33719                Kucing pasir   
33720       Kabupaten Probolinggo   

                                              paragraphs  
33713  [{'context': 'Ernest Douwes Dekker wafat dini ...  
33714  [{'context': 'Pada tanggal 18 Februari 2008, d...  
33715  [{'context': 'Sebagai tindak lanjut Atlantic C...  
33716  [{'context': 'Dia dipenjarakan di Puri Dragsho...  
33717  [{'context': 'Lahir di Sheffield, South Yorksh...  
33718  [{'context': 'John Fitzgerald Kennedy lahir di...  
33719  [{'context': 'Kucing pasir (Felis margarita), ...  
33720  [{'context': 'Sebagai salah satu kabupaten. Pr...  
(5702, 2)
                  title                                         paragraphs
0  Kristoforus Kolumbus  [{'qas': [{'question': 'S

In [46]:
def print_all_contexts_questions_answers(df):
    n_question = 0    
    for taken_topic_idx in range(df.shape[0]):
        for taken_context_idx in range(len(df.iloc[taken_topic_idx]['paragraphs'])):
            paragraph = df_tydiqa.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]
            print(f'({taken_topic_idx}, {taken_context_idx}): {df_tydiqa.iloc[taken_topic_idx]["title"]}\n{paragraph["context"]}')
            for qa in paragraph['qas']:
                print(qa['question'])
                print(qa['answers'][0]['text'])
                print()
                n_question += 1
    print(n_question)

print_all_contexts_questions_answers(df_tydiqa_dev)

(0, 0): Ernest Douwes Dekker
Ernest Douwes Dekker wafat dini hari tanggal 28 Agustus 1950 (tertulis di batu nisannya; 29 Agustus 1950 versi van der Veur, 2006) dan dimakamkan di TMP Cikutra, Bandung.
dimanakah  Dr. Ernest François Eugène Douwes Dekker meninggal?
28 Agustus 1950

(1, 0): Balai Kota Seoul
Pada tanggal 18 Februari 2008, desain Yoo Kerl dari iArc terpilih dalam kompetisi desain Pemerintah Metropolitan Seoul.[6] Bagian atas depan bangunan baru ini dirancang untuk menonjol dalam bentuk lengkung. Perancang Yoo merangkul tiga kata kunci yakni "tradisi, warga negara, dan masa depan" dalam rancangannya, yang mencerminkan elemen horisontal dari arsitektur tradisional tingkat rendah Korea dan penafsiran ulang dari nuansa mendalam dan lekukan atap atap.[7]
Siapa arsitek Balai Kota Seoul?
Yoo Kerl

(2, 0): Perserikatan Bangsa-Bangsa
Sebagai tindak lanjut Atlantic Charter tersebut, pada tanggal 25 April 1945, Konferensi PBB tentang Organisasi Internasional diadakan di San Francisco, 

In [33]:
def add_indonesian_answers_key(df):
    for taken_topic_idx in range(df.shape[0]):
        for taken_context_idx in range(len(df.iloc[taken_topic_idx]['paragraphs'])):
            paragraph = df.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]
            for i in range(len(paragraph['qas'])):
                qa = paragraph['qas'][i]
                answer_text = qa['answers'][i]['text']
                answer_start = qa['answers'][i]['answer_start']

                indonesian_answer_key = 'indonesian_answers'
                indonesian_answer = [
                    {
                        'text': answer_text,
                        'answer_start': answer_start
                    }
                ]
                df.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas'][i][indonesian_answer_key] = indonesian_answer

add_indonesian_answers_key(df_tydiqa)
add_indonesian_answers_key(df_tydiqa_dev)

In [36]:
print(df_tydiqa.iloc[0]['paragraphs'][0])
print()
print(df_tydiqa_dev.iloc[0]['paragraphs'][0])

{'context': 'Ernest Douwes Dekker wafat dini hari tanggal 28 Agustus 1950 (tertulis di batu nisannya; 29 Agustus 1950 versi van der Veur, 2006) dan dimakamkan di TMP Cikutra, Bandung.', 'qas': [{'answers': [{'answer_start': 45, 'text': '28 Agustus 1950'}], 'id': 'indonesian-496955121539262633-33', 'question': 'dimanakah  Dr. Ernest François Eugène Douwes Dekker meninggal?', 'indonesian_answers': [{'text': '28 Agustus 1950', 'answer_start': 45}]}]}

{'qas': [{'question': 'Siapakah yang menemuka benua Amerika ?', 'answers': [{'text': 'orang-orang Viking dari Eropa Utara', 'answer_start': 193}], 'id': 'indonesian-1906590851264221380-1', 'indonesian_answers': [{'text': 'orang-orang Viking dari Eropa Utara', 'answer_start': 193}]}], 'context': "Kolumbus bukanlah orang pertama yang tiba di Amerika, yang ia dapati sudah diduduki. Ia juga bukan orang Eropa pertama yang sampai ke benua itu karena sekarang telah diakui secara meluas bahwa orang-orang Viking dari Eropa Utara telah berkunjung ke A

In [38]:
df_tydiqa.to_json('../data/interim/tydiqa-goldp-v1.1-train-indonesian_prepared.json')
df_tydiqa_dev.to_json('../data/interim/tydiqa-goldp-v1.1-dev-indonesian_prepared.json')